In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import dependencies
import numpy as np
from numpy import loadtxt
import pandas as pd
from collections import Counter
import sqlalchemy
from sqlalchemy import create_engine, text

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.combine import SMOTEENN


# Define Features and Target and Split and Scale Data

In [4]:
# load data
df = pd.read_csv("lcms_df.csv")

# Create features
X = df.drop(columns = ["structure_id", "preferred_lcms_method"])

# Create target
y = df["preferred_lcms_method"]

In [6]:
df.columns

Index(['structure_id', 'preferred_lcms_method', 'MolWt', 'exactMolWt', 'qed',
       'TPSA', 'HeavyAtomMolWt', 'MolLogP', 'MolMR', 'FractionCSP3',
       'NumValenceElectrons', 'MaxPartialCharge', 'MinPartialCharge',
       'FpDensityMorgan1', 'BalabanJ', 'BertzCT', 'HallKierAlpha', 'Ipc',
       'Kappa2', 'LabuteASA', 'PEOE_VSA10', 'PEOE_VSA2', 'SMR_VSA10',
       'SMR_VSA4', 'SlogP_VSA2', 'SlogP_VSA6', 'MaxEStateIndex',
       'MinEStateIndex', 'EState_VSA3', 'EState_VSA8', 'HeavyAtomCount',
       'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
       'NumAliphaticHeterocycles', 'NumAliphaticRings',
       'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings',
       'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds',
       'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles',
       'NumSaturatedRings', 'RingCount'],
      dtype='object')

In [4]:
X.describe()

,MolWt,exactMolWt,qed,TPSA,HeavyAtomMolWt,MolLogP,MolMR,FractionCSP3,NumValenceElectrons,MaxPartialCharge,...,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount
count,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,...,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000,1048.000000
mean,388.563602,388.127765,0.620003,70.103597,369.037449,3.943946,104.416344,0.275646,142.206107,0.283503,...,1.548664,3.201336,5.005725,1.243321,7.507634,4.138359,0.044847,0.323473,0.368321,3.959924
std,83.193165,83.027644,0.172518,21.376005,79.287778,1.249958,22.610226,0.143889,29.787716,0.100816,...,0.761739,0.857863,1.262177,0.837831,2.115107,1.904837,0.228972,0.562555,0.594329,0.852166
min,226.283000,226.121846,0.138213,16.130000,212.171000,0.404920,62.030700,0.000000,80.000000,0.036113,...,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,2.000000
25%,324.652250,324.185915,0.479639,55.630000,307.964750,3.069180,87.391475,0.187500,122.000000,0.225395,...,1.000000,3.000000,4.000000,1.000000,6.000000,3.000000,0.000000,0.000000,0.000000,3.000000
50%,374.331000,373.620195,0.648190,67.350000,355.229000,3.944680,101.610650,0.277778,138.000000,0.255791,...,2.000000,3.000000,5.000000,1.000000,7.000000,4.000000,0.000000,0.000000,0.000000,4.000000
75%,447.367250,446.356872,0.767278,82.015000,427.326000,4.747525,116.444975,0.360909,160.000000,0.348704,...,2.000000,4.000000,6.000000,2.000000,9.000000,5.000000,0.000000,1.000000,1.000000,4.000000
max,774.895000,774.238214,0.925780,164.500000,737.599000,7.060600,200.318200,0.882353,286.000000,0.585809,...,4.000000,5.000000,13.000000,4.000000,16.000000,12.000000,3.000000,4.000000,4.000000,10.000000


In [5]:
# Check balance of target values
y.value_counts()

Xbridge HpH    729
Gemini LpH     319
Name: preferred_lcms_method, dtype: int64

In [6]:
# Normal train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [7]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[ 0.59733689,  0.58882916,  0.92822901, ...,  1.15766035,
         1.01840514, -1.11432097],
       [ 0.11735691,  0.10851085,  0.5338891 , ...,  1.15766035,
         1.01840514,  0.01727629],
       [-0.37292342, -0.3724427 , -0.34769352, ..., -0.56894503,
        -0.61519984,  0.01727629],
       ...,
       [-0.77102791, -0.77100915,  0.55035922, ..., -0.56894503,
        -0.61519984,  0.01727629],
       [-1.2995918 , -1.29924377,  1.29041326, ...,  1.15766035,
         1.01840514, -1.11432097],
       [-0.08062311, -0.07826255,  1.40469134, ...,  1.15766035,
         1.01840514, -1.11432097]])

# Test Machine Learning Models

In [8]:
ml_list1 = {
    "Balanced Random Forest Classifier":BalancedRandomForestClassifier(n_estimators=100, random_state=1),
    "Easy Ensemble AdaBoost Classifier":EasyEnsembleClassifier(n_estimators=100, random_state=1),
    "XGBoost Classifier":XGBClassifier(n_estimators = 100, random_state = 1)
     }


ml_list2 = {"Logistic Regression with Random Oversampling":RandomOverSampler(random_state = 1),
            "Logistic Regression with SMOTE Oversampling":SMOTE(random_state = 1, sampling_strategy = "auto"),
            "Logistic Regression with Random Undersampling":RandomUnderSampler(random_state = 1),
            "Logistic Regression with Cluster Centroids Undersampling":ClusterCentroids(random_state = 1),
           "Logistic Regression with SMOTEENN Combination Over- and Undersampling":SMOTEENN(random_state=1)}
    

ret = []

for x in ml_list1:
    ml = ml_list1[x].fit(X_train_scaled, y_train)
    y_pred = ml.predict(X_test_scaled)
    ba_score = balanced_accuracy_score(y_test, y_pred)
    ret.append({
        "Name": x,
        "Balanced Accuracy Score":ba_score,
    })

for x in ml_list2:
    X_resampled, y_resampled = ml_list2[x].fit_resample(X_train_scaled, y_train)
    ml = LogisticRegression(random_state=1).fit(X_resampled, y_resampled)

    y_pred = ml.predict(X_test_scaled)
    
    ba_score = balanced_accuracy_score(y_test, y_pred)
    ret.append({
        "Name": x,
        "Balanced Accuracy Score":ba_score,
    })
    

[00:47:45] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [9]:
summary_df = pd.DataFrame(ret).sort_values("Balanced Accuracy Score", ascending=False)
summary_df

,Name,Balanced Accuracy Score
0,Balanced Random Forest Classifier,0.873397
6,Logistic Regression with Cluster Centroids Und...,0.872631
2,XGBoost Classifier,0.865803
4,Logistic Regression with SMOTE Oversampling,0.864270
3,Logistic Regression with Random Oversampling,0.860577
7,Logistic Regression with SMOTEENN Combination ...,0.858835
5,Logistic Regression with Random Undersampling,0.856884
1,Easy Ensemble AdaBoost Classifier,0.852425
